In [32]:
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# def implicit_euler(y0, h, f):
    

In [3]:
def fixed_point_iteration(yn, yn_plus1_init, h, f, integrator, RTOL, ATOL):
    y_k = yn_plus1_init
    repeat = 1
    i = 0
    while(repeat):
        i += 1
        print("\nFixed point iteration ", i)
        y_kplus1 = integrator(yn, y_k, h, f)
        if abs(y_kplus1 - y_k) <= RTOL*y_k + ATOL:
            repeat = 0
            return y_kplus1
        y_k = y_kplus1
    


def implicit_stepper(y0, h, f, t0, tf, integrator, RTOL, ATOL):
    N    = int((tf - t0) / h)
    time = np.arange(t0, N*h, h)
    
    y    = np.zeros(N)
    y[0] = y0 # I.C.
    
    for n in range(N-1):
        y[n+1] = y[n] + h * f(y[n]) # 1 Euler step to compute an initial y_nplus1_k for fixed-point-iteration
        y[n+1] = fixed_point_iteration(y[n], y[n+1], h, f, integrator, RTOL, ATOL)

    return time, y
    

In [36]:
from scipy.sparse import diags
from scipy.linalg import lu

def implicit_euler_parabolicPde(x, ic, bc, t0, tf, dt, additional_term=0):
    # x: space discretizing array, ic: initial conditions u(0,x), bc: boundary conditions e.g. u(t,0), u(t,N)
    # t0, tf: start/end time of simulation, additional_term: optional if term is added on RHS
    
    n    = len(x) # number of space dicretizing points
    dx   = (x[-1] - x[0]) / (n-1) # space stepsize
    time = np.arange(t0, tf+dt, dt) # time discretizing array (lines)
    un   = np.zeros([len(time), len(x)]) # initialize array for solutions
    un[0]= ic # fill solution column of initial time with I.C. given
    
    # Entries in tridiagonal matrix A corresponding to the stencil weights of 1D implicit Euler
    c = dt / (dx**2)
    d = 1 + 2*c

    # Build tridiagonal matrix with stencil weights
    k      = np.array([-c*np.ones(n-1),d*np.ones(n),-c*np.ones(n-1)])
    offset = [-1,0,1]
    A      = diags(k,offset).toarray() # Stencil weight matrix
        
    # LU decomposition of A in order to solve system of linear equation
    P, L, U = lu(A)
#     np.allclose(A - p @ l @ u, np.zeros((n, n))) # returns True is P*L*U = A
    
    
    for t in range(0, len(time)):
        b       = un[t] + c * bc + additional_term # RHS
        # Solve system of linear equation for next time point u_n+1 using the lower and upper triangular matrix of A
        c       = np.linalg.solve(L, P@b) # fwd subst. 
        un[t+1] = np.linalg.solve(U, c)      # bwd subst.
    
    
    return un

In [8]:
from scipy.sparse import diags

dt = 0.001
dx = 0.1
c = dt / (dx**2)
d = 1 + 2*c

n = 5
k = np.array([-c*np.ones(n-1),d*np.ones(n),-c*np.ones(n-1)])
offset = [-1,0,1]
A = diags(k,offset).toarray()

In [25]:
from scipy.linalg import lu
n = np.shape(A)[0]
P, L, U = lu(A)
np.allclose(A - p @ l @ u, np.zeros((n, n)))

True

In [16]:
N = 4
leftBound = 0.0; rightBound = 1.0;
dx = (rightBound - leftBound) / N
x = np.arange(0, 1+dx, dx)

In [17]:
x

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [20]:
u0     = np.zeros(len(x)) # I.C.
bc     = np.zeros(len(x)) # B.C.
bc[0]  = 0
bc[-1] = 0

In [21]:
b = u0 + c * bc + np.cos(2 * np.pi * x)

In [30]:
un

array([ 8.33333333e-01,  3.49370882e-17, -8.33333333e-01, -1.74694044e-16,
        8.33333333e-01])